# 글로브(GloVe)
reference https://wikidocs.net/22885

In [2]:
from glove import Corpus, Glove
from gensim.models import KeyedVectors
import re
import nltk
nltk.download('punkt')
from nltk import sent_tokenize, word_tokenize
from lxml import etree

corpus = Corpus()

targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF-8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져옴
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통하여 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행
result = [word_tokenize(sentence) for sentence in normalized_text]

# 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬 생성
corpus.fit(result, window=5)
glove = Glove(no_components=100, learning_rate=0.05)

# 학습에 이용할 스레드의 개수는 4로 설정, 에포크는 20.
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kwon_notebook\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


학습이 완료됨

In [3]:
print(glove.most_similar('man'))

[('woman', 0.9640719879419696), ('guy', 0.8818836374295559), ('girl', 0.8765638951120126), ('kid', 0.8439685808636391)]


In [4]:
print(glove.most_similar('boy'))

[('girl', 0.9399887205245915), ('woman', 0.8554426843688894), ('man', 0.8401107819984592), ('kid', 0.8361809112584776)]


In [5]:
print(glove.most_similar('university'))

[('harvard', 0.8960422570238128), ('mit', 0.8624370326163452), ('stanford', 0.8439557706470072), ('cambridge', 0.8421337766106579)]


In [6]:
print(glove.most_similar('water'))

[('air', 0.8458107397242466), ('clean', 0.8426650677921109), ('fresh', 0.831751190658794), ('food', 0.8155714495256721)]


In [7]:
print(glove.most_similar("physics"))

[('chemistry', 0.8946326719013055), ('economics', 0.8870988871967064), ('mathematics', 0.8665281526010955), ('beauty', 0.8646533561066604)]


In [8]:
print(glove.most_similar("muscle"))

[('tissue', 0.846216604698914), ('nerve', 0.8193104189398185), ('channel', 0.7650735834396339), ('skeletal', 0.7645080426800865)]
